Social Media Analytics Assignment #1

Siraj Hatoum,
Mohamad Khalili,
Mehdi Yachfine,
Hirad Nourbakhsh,
Brock Beckley

Part 1 - Find Predictors of Influence

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

Importing the Dataset

In [4]:
df = pd.read_csv('train.csv')
df = df.dropna()
df.head()

,Choice,A_follower_count,A_following_count,A_listed_count,A_mentions_received,A_retweets_received,A_mentions_sent,A_retweets_sent,A_posts,A_network_feature_1,...,B_following_count,B_listed_count,B_mentions_received,B_retweets_received,B_mentions_sent,B_retweets_sent,B_posts,B_network_feature_1,B_network_feature_2,B_network_feature_3
0,0,228,302,3,0.583979,0.100503,0.100503,0.100503,0.362150,2,...,29808,1689,15.430498,3.984029,8.204331,0.332423,6.988815,66,75.530303,1916.893939
1,0,21591,1179,228,90.456506,25.798292,5.709329,1.111159,5.176620,369,...,848,1610,40.495021,8.943607,3.227677,0.564343,1.070321,163,132.030675,2931.515337
2,0,7310,1215,101,25.503644,9.556347,5.361519,0.591206,3.589718,95,...,482,206,0.734696,0.354379,0.603202,0.100503,0.603202,3,10.333333,277.333333
3,0,20,7,2,7.690824,0.277306,1.331508,0.100503,2.830627,6,...,17637,278,572.874856,390.293681,27.552040,7.167557,32.101906,1762,23.469296,1395.845634
4,1,45589,862,2641,148.854279,36.998884,27.881768,3.333492,23.861282,551,...,1711,181,21.601866,3.581661,6.764657,1.119727,4.563246,85,48.500000,1993.627907


Feature Engineering

In this section, we are creating new features based on the features available in the dataset. Since the goal is to identify whether influencer A is more influent than influencer B, instead of having individual metrics for both influencers, we create comparison metrics. For instance, instead of having the individual follower count for each influencer, we have the difference in followers count between influencers A and B. The ratio could have also been calculated, but we chose to go with the difference, to avoid potential divisions by 0.

We tried engineering more advanced features, such as the level of interaction, defined by the sum of retweets and mentions, divided by the total number of followers for a given influencer. However, as will be shown further down, those features did not have a significant importance in predicting influence.

In [5]:
#follower count difference
df['Follower Count Difference'] = df['A_follower_count'] - df['B_follower_count'] 
#following count difference
df['Followig Count Difference'] = df['A_following_count'] - df['B_following_count'] 
#listed count difference
df['Listed Count Difference'] = df['A_listed_count'] - df['B_listed_count'] 
#received retweets difference
df['Received Retweets Difference'] = df['A_retweets_received'] - df['B_retweets_received'] 
#received mention difference
df['Received Mentions Difference'] = df['A_mentions_received'] - df['B_mentions_received']
#sent retweets difference
df['Sent Retweets Difference'] = df['A_retweets_sent'] - df['B_retweets_sent'] 
#sent mentions difference
df['Sent Mentions Difference'] = df['A_mentions_sent'] - df['B_mentions_sent'] 
#post difference
df['Post Difference'] = df['A_posts'] - df['B_posts']

###################################################################################################################
#Ratio of interactions (retweets received + mentions received)/posts -- NOT VERY USEFUL
#df['Interaction Ratio A'] = (df['A_retweets_received'] + df['A_mentions_received'])/df['A_follower_count']
#df['Interaction Ratio B'] = (df['B_retweets_received'] + df['B_mentions_received'])/df['B_follower_count']

#Listed/follower ratio --- NOT VERY USEFUL
#df['LF Ratio A'] = df['A_listed_count']/df['A_follower_count']
#df['LF Ratio B'] = df['B_listed_count']/df['B_follower_count']

#ratio of infulencer's posts that are retweeted ---- NOT VERY USEFUL
#df['Retweet % A'] = df['A_retweets_received']/df['A_posts']
#df['Retweet % B'] = df['B_retweets_received']/df['B_posts']

#Calculating following/followers ratio for each influencer ------ NOT VERY USEFUL
#df['following/followers_A'] = df['A_following_count']/df['A_follower_count']
#df['following/followers_B'] = df['B_following_count']/df['B_follower_count']

In [6]:
#Dropping the features that were precursors to the newly designed features:

df = df.drop(columns = ['A_follower_count',
                   'A_following_count',
                   'A_listed_count',
                   'A_mentions_received',
                   'A_retweets_received',
                   'A_mentions_sent',
                   'A_retweets_sent',
                   'A_posts', 
                   'B_follower_count',
                   'B_following_count',
                   'B_listed_count',
                   'B_mentions_received',
                   'B_retweets_received',
                   'B_mentions_sent',
                   'B_retweets_sent',
                   'B_posts'])
df.head()

,Choice,A_network_feature_1,A_network_feature_2,A_network_feature_3,B_network_feature_1,B_network_feature_2,B_network_feature_3,Follower Count Difference,Followig Count Difference,Listed Count Difference,Received Retweets Difference,Received Mentions Difference,Sent Retweets Difference,Sent Mentions Difference,Post Difference
0,0,2,166.500000,11355.000000,66,75.530303,1916.893939,-34235,-29506,-1686,-3.883525,-14.846518,-0.231920,-8.103828,-6.626665
1,0,369,18.442971,1330.366048,163,132.030675,2931.515337,-17671,331,-1382,16.854685,49.961485,0.546816,2.481652,4.106299
2,0,95,68.927835,5999.896907,3,10.333333,277.333333,3688,733,-105,9.201969,24.768949,0.490702,4.758317,2.986516
3,0,6,2.000000,96.166667,1762,23.469296,1395.845634,-19542,-17630,-276,-390.016375,-565.184032,-7.067053,-26.220532,-29.271279
4,1,551,127.404293,2833.847943,85,48.500000,1993.627907,38035,-849,2460,33.417223,127.252413,2.213765,21.117111,19.298035


Feature Importance

In order to compute the relative important of each feature in our newly engineered dataset, an initial random forest model was run. That way, we have a quantitative way of assessing the importance of each predictor, which will reinforce/confirm domain knowledge when identifying the most important predictors of influence.

In [7]:
X = df.drop(columns = 'Choice')
y = df['Choice']

scaler = StandardScaler()
X_std = scaler.fit_transform(X)

In [8]:
rf = RandomForestClassifier(random_state = 0, n_estimators = 500)
model = rf.fit(X_std,y)

model.feature_importances_

importances = pd.DataFrame(list(zip(X.columns,model.feature_importances_)), columns = ['predictor','feature importance'])

importances.sort_values(by = 'feature importance', ascending = False)

,predictor,feature importance
8,Listed Count Difference,0.148427
6,Follower Count Difference,0.121338
10,Received Mentions Difference,0.110956
9,Received Retweets Difference,0.099928
3,B_network_feature_1,0.070874
0,A_network_feature_1,0.069579
13,Post Difference,0.055813
12,Sent Mentions Difference,0.054854
7,Followig Count Difference,0.053043
2,A_network_feature_3,0.044415


Training and testing the model

In [9]:
from sklearn.model_selection import train_test_split

X = df[['Listed Count Difference',
        'Follower Count Difference',
        'Received Mentions Difference',
        'Received Retweets Difference',
        'B_network_feature_1',
        'A_network_feature_1',
        'Post Difference']]

y = df['Choice']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.05, random_state=5)

X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

model = rf.fit(X_train_std,y_train)
y_pred = model.predict(X_test_std)

print(round(accuracy_score(y_pred,y_test),3)*100,'% accuracy')

83.3 % accuracy


Financial Analysis

In [11]:
#Probability of follower purchasing product
non_influencer = 0
influencer_tweet = 0.0002 #percent of followers purchasing with one tweet
influencer_tweet2 = 0.0003 #percent of followers purchasing with two tweets

In [12]:
#Sales & Cost Info
profitmargin = 10 #One customer can only buy one unit
tweet_cost = 5 #Without analytics for one tweet
tweet_cost_analytics = 10 #With analytics for two tweets
tweet_cost_noninf = 0 #Tweet cost for non-influencer

In [13]:
wrong_observations = []

for i in range(len(y_pred)):
    if (y_test.iloc[i] != y_pred[i]):
        wrong_observations.append(y_test.index[i])
wrong_observations

[3552,
 1434,
 4005,
 2060,
 3941,
 2614,
 767,
 3372,
 674,
 2983,
 5031,
 2693,
 5437,
 769,
 3994,
 5415,
 3409,
 4393,
 1752,
 2191,
 2165,
 892,
 1977,
 608,
 5426,
 51,
 4134,
 1722,
 3817,
 4488,
 4237,
 2928,
 4150,
 4619,
 729,
 3836,
 3178,
 2509,
 944,
 3680,
 2892,
 1110,
 1781,
 40,
 532,
 111]

In [14]:
correct_observations = []

for i in range(len(y_pred)):
    if (y_test.iloc[i] == y_pred[i]):
        correct_observations.append(y_test.index[i])
correct_observations

[4496,
 4938,
 1183,
 453,
 2904,
 5003,
 1773,
 2582,
 3721,
 3958,
 796,
 3795,
 5422,
 1538,
 586,
 5216,
 375,
 4355,
 814,
 3940,
 4228,
 4499,
 578,
 468,
 3867,
 2953,
 2043,
 2068,
 2951,
 563,
 4754,
 2290,
 1207,
 646,
 2641,
 1323,
 4545,
 4421,
 4874,
 5180,
 4208,
 5284,
 1546,
 3856,
 4749,
 2261,
 498,
 3513,
 3682,
 4384,
 4872,
 3495,
 4216,
 4152,
 4973,
 33,
 1843,
 958,
 2851,
 938,
 4196,
 3741,
 559,
 1172,
 4201,
 4410,
 2435,
 2298,
 4017,
 4831,
 3834,
 5301,
 1708,
 2469,
 2708,
 2084,
 2527,
 1872,
 1914,
 5359,
 2717,
 4086,
 5363,
 3675,
 3028,
 1641,
 1761,
 4742,
 5170,
 1293,
 2732,
 1719,
 2720,
 5338,
 261,
 279,
 4212,
 4272,
 4953,
 1134,
 482,
 1276,
 4026,
 3186,
 2931,
 3407,
 3282,
 1176,
 4767,
 1349,
 1936,
 4267,
 424,
 564,
 3279,
 5054,
 4633,
 3565,
 2946,
 3055,
 3655,
 265,
 651,
 1848,
 3362,
 3267,
 2908,
 5268,
 5238,
 656,
 1847,
 3743,
 569,
 728,
 2153,
 5499,
 2695,
 2328,
 4398,
 4807,
 5187,
 4407,
 318,
 4739,
 1528,
 5321,
 347

In [15]:
test_observations = correct_observations + wrong_observations

In [17]:
df1 = pd.read_csv('train.csv')

In [18]:
df1 = df1[df1.index.isin(test_observations)]
df1.head()

,Choice,A_follower_count,A_following_count,A_listed_count,A_mentions_received,A_retweets_received,A_mentions_sent,A_retweets_sent,A_posts,A_network_feature_1,...,B_following_count,B_listed_count,B_mentions_received,B_retweets_received,B_mentions_sent,B_retweets_sent,B_posts,B_network_feature_1,B_network_feature_2,B_network_feature_3
33,1,130850,3394,449,92.869651,16.538484,2.548048,0.100503,3.036688,355,...,431,44,0.555204,0.100503,0.100503,0.100503,0.595962,2,836.000000,11447.500000
40,0,627949,150866,6820,246.403846,74.705430,21.260939,1.320681,15.278066,683,...,6,2963,98.954241,27.797596,0.796262,0.332423,0.332423,430,241.680460,8490.301149
51,1,232753,320,7919,175.264776,100.480820,3.108490,1.614717,4.401919,679,...,293,970,1729.451634,763.659739,30.829977,8.568367,93.311571,4067,12.310269,1636.379704
64,1,1352348,331,15647,19129.848662,14552.733991,2.065418,0.580955,16.281502,52580,...,673,747,55.993546,22.321945,6.946233,0.341936,6.503977,202,152.495098,3286.617647
67,0,49326,42259,1446,0.452340,0.275538,0.100503,0.100503,0.100503,2,...,4524,11946,6782.405338,2944.524245,12.955723,2.791861,66.206208,21678,6.859364,1113.202016


In [19]:
#Without Analytics
df1['Expected_Return_No_Analytics'] = ''
df1.loc[df1["Choice"] == 1, 'Expected_Return_No_Analytics'] = (df1["A_follower_count"] * influencer_tweet)*profitmargin - tweet_cost*2
df1.loc[df["Choice"] == 0, 'Expected_Return_No_Analytics'] = (df1["B_follower_count"] * influencer_tweet)*profitmargin - tweet_cost*2
Total_noanalytics = df1["Expected_Return_No_Analytics"].sum()
print("Total Expected Return without analytics is $", round(Total_noanalytics,2))

Total Expected Return without analytics is $ 653549.22


In [20]:
#Perfect Model
df1['Expected_Return_Perfect'] = ''
df1.loc[df["Choice"] == 1, 'Expected_Return_Perfect'] = (df1["A_follower_count"] * influencer_tweet2)*profitmargin - tweet_cost_analytics
df1.loc[df["Choice"] == 0, 'Expected_Return_Perfect'] = (df1["B_follower_count"] * influencer_tweet2)*profitmargin - tweet_cost_analytics
Total_perfect = df1["Expected_Return_Perfect"].sum()
print("Total Expected Return with perfect analytics is $", round(Total_perfect,2))
print("The increase in net profit from using a perfect analytic model is $",round(Total_perfect - Total_noanalytics,2))

Total Expected Return with perfect analytics is $ 981698.82
The increase in net profit from using a perfect analytic model is $ 328149.61


In [21]:
###With Analytics
df1['Expected_Return_Analytics'] = df1['Expected_Return_Perfect']
df1.loc[df1.index.isin(wrong_observations), 'Expected_Return_Analytics'] = -tweet_cost_analytics
df1.head()

,Choice,A_follower_count,A_following_count,A_listed_count,A_mentions_received,A_retweets_received,A_mentions_sent,A_retweets_sent,A_posts,A_network_feature_1,...,B_retweets_received,B_mentions_sent,B_retweets_sent,B_posts,B_network_feature_1,B_network_feature_2,B_network_feature_3,Expected_Return_No_Analytics,Expected_Return_Perfect,Expected_Return_Analytics
33,1,130850,3394,449,92.869651,16.538484,2.548048,0.100503,3.036688,355,...,0.100503,0.100503,0.100503,0.595962,2,836.000000,11447.500000,251.7,382.55,382.55
40,0,627949,150866,6820,246.403846,74.705430,21.260939,1.320681,15.278066,683,...,27.797596,0.796262,0.332423,0.332423,430,241.680460,8490.301149,106.18,164.27,-10
51,1,232753,320,7919,175.264776,100.480820,3.108490,1.614717,4.401919,679,...,763.659739,30.829977,8.568367,93.311571,4067,12.310269,1636.379704,455.506,688.259,-10
64,1,1352348,331,15647,19129.848662,14552.733991,2.065418,0.580955,16.281502,52580,...,22.321945,6.946233,0.341936,6.503977,202,152.495098,3286.617647,2694.696,4047.044,4047.044
67,0,49326,42259,1446,0.452340,0.275538,0.100503,0.100503,0.100503,2,...,2944.524245,12.955723,2.791861,66.206208,21678,6.859364,1113.202016,4537.742,6811.613,6811.613


In [22]:
Total_model = df1["Expected_Return_Analytics"].sum()
print("Total Expected Return with analytics is $", round(Total_model,2))
print("The increase in net profit from using an analytic model is $",round(Total_model - Total_noanalytics,2))

Total Expected Return with analytics is $ 932271.51
The increase in net profit from using an analytic model is $ 278722.29


Part 2: Finding Influencers from Twitter

Tweet Scraping

In [ ]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import matplotlib.pyplot as plt
# NLTK VADER for sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import requests
from datetime import datetime, timedelta
import requests
import pandas as pd
import re

from datetime import datetime, timedelta
import requests
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import datetime, timedelta
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAANeNZAEAAAAAWqEd3GXb3R8V%2FdhbCITAL%2BHA9RM%3D32FXQSrOUqzLm0LSmsKnq75B9CwIZLtsH3OAoG8HsgyC08Jrra'

In [ ]:
# setup the API request
ref= 'https://api.twitter.com/2/tweets/search/recent'

headers= {'authorization': f'Bearer {BEARER_TOKEN}'}
params= {
    'query': '(Champions league OR UCL) (lang:en)',
    'max_results': '100',
    'tweet.fields': 'created_at,lang,author_id',
    #'expansions': 'entities.mentions.username',
    #'user.fields': 'username'
}


In [ ]:
#Function to get data from the requests
def get_data(tweet):
    data = {'id':tweet['id'],
            'author_id':tweet['author_id'],
            #'username':tweet['entities']['mentions'][0]['username'],
            #'username_id': tweet['entities']['mentions'][0]['id'],
        'created_at':tweet['created_at'],
        'text':tweet['text']}
    return data

#tweet['entities']['mentions'][0]['username']

In [ ]:
dtformat = '%Y-%m-%dT%H:%M:%SZ'  # Setting the date format required by twitter API

In [ ]:
# function that subtracts 60 min from the current datetime
def time_slots(end_date, mins):
    date = datetime.strptime(end_date, dtformat)
    back_in_time = date - timedelta(minutes=mins)
    return back_in_time.strftime(dtformat)

#date = '2022-03-16T23:55:55Z'


In [ ]:
#Defining current hour
date = '16/03/22 23:59:00'
date = datetime.strptime(date, '%d/%m/%y %H:%M:%S') # get current datetime

last_week = date - timedelta(days=2)  # datetime 7 days ago
date = date.strftime(dtformat)  # convert current datetime to format for twitter API

In [ ]:
df = pd.DataFrame()  # initialize dataframe

In [ ]:
while True:
    
    if (datetime.strptime(date, dtformat) < last_week):
        # break the loop when we reach one week ago
        break
    
    go_back = time_slots(date, 55)  # get 60 minutes before 'now'
    
    # setting starting and end time for the API
    params['start_time'] = go_back
    params['end_time'] = date
    
    #Sending request
    response = requests.get(ref,
                            params=params,
                            headers=headers)
    
    date = go_back  # moving back the window by 60 min
    
    # Adding the tweets data to the dataframe
    for tweet in response.json()['data']:
        row = get_data(tweet)  # we defined this function earlier
        df = df.append(row, ignore_index=True)

In [ ]:
#converting 'created_at' to date time
df.created_at = pd.to_datetime(df['created_at'])

df.to_csv('C:/Users/Mehdi/Desktop/tweets_ucl_noUsername.csv')

In [ ]:
from twitterUsernameviaUserID import getHandles as gH
temp = gH.getHandles(df.author_id.tolist(), 10)

id_to_username_df = pd.DataFrame(list(temp.items()), columns=['author_id', 'username'])
id_to_username_df.to_csv('C:/Users/Mehdi/Desktop/id_to_username_df.csv')

In [ ]:
df_final = pd.merge(df,id_to_username_df, how='left', on='author_id')

#Check
print(df_final.isnull().sum())

df_final.to_csv('C:/Users/Mehdi/Desktop/final_tweets_df.csv')

Creating Tweet Table

In [ ]:
df = pd.read_csv('/Users/hiradnourbakhsh/Desktop/INSY 670/final_tweets_df.csv')
df

In [ ]:
df = df[['username', 'text']]
df

In [ ]:
df = df.rename(columns = {'username': 'handle'})
df

Second dataframe: retweets

In [ ]:
col_2_keys = []
col_2_values = []
for i in range(len(df.text)):
    if 'RT' in df.text[i]:
        col_2_keys.append(df.text[i].split(' ')[1])
        col_2_values.append(df.text[i].split(' ')[1])
        col_2_keys.append(df.handle[i])
        col_2_values.append(df.text[i].split(' ')[1])
#col_2_keys
#col_2_values

col_2_dict = dict(col1 = col_2_keys, col2 = col_2_values)
col_2_dict

In [ ]:
rt = pd.DataFrame(col_2_dict)
rt

In [ ]:
col3 = []
for i in range(len(rt)):
    if rt.col1[i] == rt.col2[i]:
        col3.append('Tweet')
    elif rt.col1[i] != rt.col2[i]:
        col3.append('Retweet')
rt['col3'] = col3
rt

In [ ]:
str1 = 'abc'

In [ ]:
str1 = '@' + str1
str1

In [ ]:
for i in range(len(rt.col1)):
    if '@' not in rt.col1[i]:
        rt.col1[i] = '@' + rt.col1[i]
rt.col1

In [ ]:
for i in range(len(rt.col1)):
    if ':' in rt.col1[i]:
        rt.col1[i] = rt.col1[i].replace(':', '')
rt.col1

In [ ]:
for i in range(len(rt.col2)):
    if ':' in rt.col2[i]:
        rt.col2[i] = rt.col2[i].replace(':', '')
rt.col2

In [ ]:
rt

In [ ]:
rt.to_csv('/Users/hiradnourbakhsh/Desktop/INSY 670/Tweet_table.csv', index = False)

Generating the Network Diagram

In [ ]:
import pandas as pd
import networkx as nx

In [ ]:
directory = '../data/'

In [ ]:
network_df = pd.read_csv(directory+'Tweet_table.csv')
network_df.columns = ['user1', 'user2', 'type']

In [ ]:
network_df.head()

In [ ]:
rt_in = network_df.type=='Retweet'
rts = network_df[rt_in]
tweets = network_df.drop(index= rts.index)

In [ ]:
graph = nx.DiGraph()

In [ ]:
graph.add_edges_from(zip(rts.user1, rts.user2), ttype = 'RT')
graph.add_edges_from(zip(tweets.user1, tweets.user2), ttype = 'tweet')

In [ ]:
nx.draw(graph, with_labels = True)

In [ ]:
graph_degree = pd.DataFrame(graph.degree()).sort_values(by=1, ascending = False)

In [ ]:
pd.Series(nx.algorithms.centrality.betweenness.betweenness_centrality(graph)).sort_values()

In [ ]:
pd.Series(nx.algorithms.centrality.closeness.closeness_centrality(graph)).sort_values(ascending=False)

In [ ]:
degree = pd.Series(dict(graph.degree))

In [ ]:
retweets_res = network_df[network_df.type == "Retweet"].groupby('user2').aggregate({'type':'count'})

In [ ]:
retweets_res

In [ ]:
degree.join()